In [ ]:
import cv2
import numpy as np
from random import shuffle
import os
from tqdm import tqdm

traindir = "/Users/Sreeram/Downloads/DogsvsCats/train"
testdir = "/Users/Sreeram/Downloads/DogsvsCats/test"
imgsize = 1000
lr = 1e-3

modelname = "dogsvscats-{}-{}.model".format(lr, "8conv-basic")

def label_image(img):
    wordlabel  = img.split(".")[-3]
    if wordlabel == "cat":
        return [1, 0]
    elif wordlabel == "dog":
        return [0, 1]
    
def create_train_data():
    traindata = []
    for img in tqdm(os.listdir(traindir)):
        label = label_image(img)
        path = os.path.join(traindir, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (imgsize, imgsize))
        traindata.append([np.array(img), np.array(label)])

    shuffle(traindata)
    np.save("train_data.npy", traindata)
    return traindata

def process_test_data():
    testingdata = []
    for img in tqdm(os.listdir(testdir)):
        path = os.path.join(testdir, img)
        img_num = img.split(".")[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (imgsize, imgsize))
        testingdata.append([np.array(img), img_num])
    
    np.save("test_data.npy", testingdata)
    return testingdata

train_data = create_train_data()

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


import tensorflow as tf
tf.reset_default_graph()


convnet = input_data(shape=[None, imgsize, imgsize, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir="log")

if os.path.exists("{}.meta".format(modelname)):
    model.load(modelname)
    print("Model Loaded!")

train = train_data
test = train_data

X = np.array([i[0] for i in train]).reshape(-1, imgsize, imgsize, 1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1, imgsize, imgsize, 1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=modelname)

model.save(modelname)

import matplotlib.pyplot as plt

test_data = process_test_data()

fig = plt.figure()


for num, data in enumerate(test_data):
    
    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(100, 5, num+1)
    orig = img_data
    data = img_data.reshape(imgsize, imgsize, 1)
    
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1:
        str_label = "Dog"
    
    else:
        str_label = "Cat"
    
    y.imshow(orig, cmap="gray")
    plt.title(str_label)
    
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
    
plt.show()

with open("Submission-file.csv", "w") as f:
    f.write("id, label\n")
    
with open("Submission-file.csv", "a") as f:
    for data in tqdm(test_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data
        data = img_data.reshape(imgsize, imgsize, 1)
        model_out = model.predict([data])[1]
        f.write("{}, {}\n".format(img_num, model_out[1]))

 33%|███▎      | 8210/25000 [01:05<09:30, 29.43it/s] 